In [1]:
!pip install -q "git+https://github.com/Keyza-asyadda/DERMIS-MachineLearning.git#egg=dermis_utils&subdirectory=utils"
!pip install -q keras-cv-attention-models

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import warnings
warnings.filterwarnings("ignore")

from tensorflow import keras
import tensorflow as tf

import keras_cv_attention_models as kecam

import dermis_utils

import os
import gc

D0615 03:33:22.462294722      13 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0615 03:33:22.462318435      13 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0615 03:33:22.462321608      13 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0615 03:33:22.462323830      13 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0615 03:33:22.462325983      13 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0615 03:33:22.462328332      13 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0615 03:33:22.462330475      13 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0615 03:33:22.

In [3]:
DATA_DIR = '../input/isic-2019-capstone/ISIC_2019'
CACHE_DIR = '../tmp/'
SAVE_DIR = os.getcwd()
IMG_SIZE = 224
EPOCHS = 40
BATCH_SIZE = 128
BRIGHTNESS_LEVEL = .2
NUM_CLASSES = 8
LR = 0.00035

In [4]:
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

In [5]:
try:
    tpu =tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
except:
    if len(tf.config.list_logical_devices("GPU")) > 1:
        strategy = tf.distribute.MirroredStrategy()
    else:
        strategy = tf.distribute.get_strategy()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

## Import dataset

### Define augmentation function

In [6]:
if len(tf.config.list_logical_devices("TPU")) >0:
    augmentation_model = keras.Sequential([
        keras.layers.RandomBrightness(BRIGHTNESS_LEVEL),
        keras.layers.RandomFlip(),
        keras.layers.RandomRotation(.9),
    ])
else:
    with strategy.scope():
        augmentation_model = keras.Sequential([
            keras.layers.RandomBrightness(BRIGHTNESS_LEVEL),
            keras.layers.RandomFlip(),
            keras.layers.RandomRotation(.9),
        ])

@tf.function
def aug_image_label(images, labels):
    aug = augmentation_model(images, training = True)
    return aug, labels

In [7]:
train_ds = keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'train'),
    image_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True,
    label_mode = "categorical"
)

Found 17731 files belonging to 8 classes.


In [8]:
class_weight = train_ds.class_names

In [9]:
# train_ds = train_ds.cache(os.path.join(CACHE_DIR, 'train'))
train_ds = train_ds.map(aug_image_label, num_parallel_calls = tf.data.AUTOTUNE, deterministic = False)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

In [10]:
val_ds = keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, 'val'),
    image_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = False, # No need to shuffle validation set, and also to reduce computation
    label_mode = 'categorical'
)

Found 3800 files belonging to 8 classes.


In [11]:
val_ds = val_ds.cache(os.path.join(CACHE_DIR, 'val'))
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

## Calculate class weight for weighted cross entropy

In [12]:
class_weight = list(map(lambda x: len(os.listdir(os.path.join(DATA_DIR, 'train', x))), class_weight))
class_weight = tf.divide(tf.reduce_sum(class_weight), class_weight)
class_weight = tf.divide(class_weight * 10, tf.reduce_sum(class_weight))
class_weight = dict(enumerate(class_weight))

## Define model

In [13]:
def get_pretrained(
    trainable_blocks = [],
    train_all = False,
):
    base = kecam.convnext.ConvNeXtV2Nano(input_shape = (IMG_SIZE, IMG_SIZE, 3), num_classes = 0, pretrained = 'imagenet')
    if not train_all:
        for layer in base.layers:
            layer.trainable = any(i in layer.name for i in trainable_blocks) and not isinstance(layer, keras.layers.BatchNormalization)
    else:
        for layer in base.layers:
            layer.trainable = not isinstance(layer, keras.layers.BatchNormalization)
    return base

In [14]:
def get_model(
    h_units = 64,
    trainable_blocks = ['stack4_block1', 'stack4_block2'],
    train_all_base_model = False
):
    inputs = keras.Input(shape = (IMG_SIZE, IMG_SIZE, 3))
    x = dermis_utils.layers.PreprocessKecam()(inputs)
    x = get_pretrained(
        trainable_blocks = trainable_blocks,
        train_all = train_all_base_model
    )(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(h_units)(x)
    x = dermis_utils.layers.Swish()(x)
    output = keras.layers.Dense(NUM_CLASSES)(x)
    return keras.Model(inputs = inputs, outputs = output)

## Train model

### Train model with Focal Loss

In [15]:
with strategy.scope():
    model = get_model()

62970120/62970120 [==============================] - 1s 0us/step
>>>> Load pretrained from: /root/.keras/models/convnext_v2_nano_imagenet.h5


In [16]:
with strategy.scope():
    
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = dermis_utils.losses.CategoricalFocalCrossentropy(gamma = 1., from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [17]:
with strategy.scope():
    
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'focal_loss_unfreeze_last_two.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'focal_loss_unfreeze_last_two/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "focal-loss-unfreeze-last-two-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [18]:
model.fit(
    train_ds,
    epochs = EPOCHS,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    validation_data = val_ds
)

Epoch 1/40


2023-06-15 03:34:41.705561: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_50/ReadVariableOp.
2023-06-15 03:34:42.159353: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_50/ReadVariableOp.


139/139 [==============================] - ETA: 0s - loss: 0.8393 - accuracy: 0.6223 - auc: 0.7376 - recall: 0.9532 - precision: 0.2293

2023-06-15 03:35:39.030082: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-15 03:35:39.240931: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 96s 372ms/step - loss: 0.8393 - accuracy: 0.6223 - auc: 0.7376 - recall: 0.9532 - precision: 0.2293 - val_loss: 0.6565 - val_accuracy: 0.6737 - val_auc: 0.8204 - val_recall: 0.9768 - val_precision: 0.2364 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 17s 110ms/step - loss: 0.6329 - accuracy: 0.6870 - auc: 0.8274 - recall: 0.9798 - precision: 0.2395 - val_loss: 0.5745 - val_accuracy: 0.7074 - val_auc: 0.8453 - val_recall: 0.9871 - val_precision: 0.2322 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 18s 113ms/step - loss: 0.5584 - accuracy: 0.7191 - auc: 0.8525 - recall: 0.9877 - precision: 0.2375 - val_loss: 0.5438 - val_accuracy: 0.7187 - val_auc: 0.8607 - val_recall: 0.9866 - val_precision: 0.2402 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 17s 111ms/step - loss: 0.5025 - accuracy: 0.7414 - auc: 0.8673 - recall: 0.9896 - precision: 0.2402 - val_loss: 0.5352 - val_accu

In [19]:
model.save(os.path.join(SAVE_DIR,"focal_loss_unfreeze_last_two"))

INFO:tensorflow:Assets written to: /kaggle/working/focal_loss_unfreeze_last_two/assets


INFO:tensorflow:Assets written to: /kaggle/working/focal_loss_unfreeze_last_two/assets


### Train model with weighted cross entropy

In [20]:
keras.backend.clear_session()
gc.collect()

218127

In [21]:
with strategy.scope():
    model = get_model()

>>>> Load pretrained from: /root/.keras/models/convnext_v2_nano_imagenet.h5


In [22]:
with strategy.scope():
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = keras.losses.CategoricalCrossentropy(from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [23]:
with strategy.scope():
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'weighted_crossentropy_loss_unfreeze_last_two.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'weighted_crossentropy_loss_unfreeze_last_two/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "weighted-crossentropy-loss-unfreeze-last-two-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [24]:
model.fit(
    train_ds,
    epochs = EPOCHS,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    class_weight = class_weight,
    validation_data = val_ds
)

Epoch 1/40


2023-06-15 03:48:41.329817: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_50/ReadVariableOp.
2023-06-15 03:48:41.802049: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_50/ReadVariableOp.


139/139 [==============================] - ETA: 0s - loss: 0.4448 - accuracy: 0.4280 - auc: 0.7700 - recall: 0.8443 - precision: 0.2179

2023-06-15 03:49:39.144394: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-15 03:49:39.395799: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 94s 353ms/step - loss: 0.4448 - accuracy: 0.4280 - auc: 0.7700 - recall: 0.8443 - precision: 0.2179 - val_loss: 1.0842 - val_accuracy: 0.6229 - val_auc: 0.8543 - val_recall: 0.9232 - val_precision: 0.2759 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 17s 108ms/step - loss: 0.3438 - accuracy: 0.5387 - auc: 0.8517 - recall: 0.9201 - precision: 0.2488 - val_loss: 1.2595 - val_accuracy: 0.5505 - val_auc: 0.8794 - val_recall: 0.9187 - val_precision: 0.2449 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 17s 109ms/step - loss: 0.3011 - accuracy: 0.5766 - auc: 0.8740 - recall: 0.9368 - precision: 0.2597 - val_loss: 1.1787 - val_accuracy: 0.5526 - val_auc: 0.8796 - val_recall: 0.9526 - val_precision: 0.2510 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 17s 108ms/step - loss: 0.2761 - accuracy: 0.6008 - auc: 0.8853 - recall: 0.9504 - precision: 0.2648 - val_loss: 1.0342 - val_accu

In [25]:
model.save(os.path.join(SAVE_DIR,"weighted_crossentropy_loss_unfreeze_last_two"))

INFO:tensorflow:Assets written to: /kaggle/working/weighted_crossentropy_loss_unfreeze_last_two/assets


INFO:tensorflow:Assets written to: /kaggle/working/weighted_crossentropy_loss_unfreeze_last_two/assets


### Train focal loss model with three last blocks being trainable

In [26]:
keras.backend.clear_session()
gc.collect()

218199

In [27]:
with strategy.scope():
    model = get_model(trainable_blocks = ['stack4_block2', 'stack4_block1', 'stack4_downsample', 'stack3_block8'])

>>>> Load pretrained from: /root/.keras/models/convnext_v2_nano_imagenet.h5


In [28]:
with strategy.scope():
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = dermis_utils.losses.CategoricalFocalCrossentropy(gamma = 1., from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [29]:
with strategy.scope():
    
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'focal_loss_unfreeze_last_three.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'focal_loss_unfreeze_last_three/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "focal-loss-unfreeze-last-three-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [30]:
model.fit(
    train_ds,
    epochs = EPOCHS,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    validation_data = val_ds
)

Epoch 1/40


2023-06-15 04:02:49.417448: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_78/ReadVariableOp.
2023-06-15 04:02:49.897253: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_78/ReadVariableOp.


139/139 [==============================] - ETA: 0s - loss: 0.8715 - accuracy: 0.6066 - auc: 0.7105 - recall: 0.9174 - precision: 0.2626

2023-06-15 04:03:53.010533: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-15 04:03:53.218302: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 101s 386ms/step - loss: 0.8715 - accuracy: 0.6066 - auc: 0.7105 - recall: 0.9174 - precision: 0.2626 - val_loss: 0.6494 - val_accuracy: 0.6824 - val_auc: 0.8109 - val_recall: 0.9684 - val_precision: 0.2698 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 17s 111ms/step - loss: 0.6522 - accuracy: 0.6835 - auc: 0.8084 - recall: 0.9634 - precision: 0.2654 - val_loss: 0.5951 - val_accuracy: 0.6987 - val_auc: 0.8513 - val_recall: 0.9776 - val_precision: 0.2640 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 18s 114ms/step - loss: 0.5627 - accuracy: 0.7156 - auc: 0.8451 - recall: 0.9777 - precision: 0.2661 - val_loss: 0.5725 - val_accuracy: 0.7203 - val_auc: 0.8495 - val_recall: 0.9761 - val_precision: 0.2619 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 17s 113ms/step - loss: 0.5163 - accuracy: 0.7324 - auc: 0.8662 - recall: 0.9794 - precision: 0.2708 - val_loss: 0.5347 - val_acc

In [31]:
model.save(os.path.join(SAVE_DIR, 'focal_loss_unfreeze_last_three'))

INFO:tensorflow:Assets written to: /kaggle/working/focal_loss_unfreeze_last_three/assets


INFO:tensorflow:Assets written to: /kaggle/working/focal_loss_unfreeze_last_three/assets


### Train weighted cross entropy model with three last blocks being trainable

In [32]:
keras.backend.clear_session()
gc.collect()

218199

In [33]:
with strategy.scope():
    model = get_model(trainable_blocks = ['stack4_block2', 'stack4_block1', 'stack4_downsample', 'stack3_block8'])

>>>> Load pretrained from: /root/.keras/models/convnext_v2_nano_imagenet.h5


In [34]:
with strategy.scope():
    optimizer = keras.optimizers.Adam(
        LR
    )
    
    model.compile(
        optimizer = optimizer,
        loss = keras.losses.CategoricalCrossentropy(from_logits = True),
        metrics = [
            "accuracy",
            keras.metrics.AUC(
                multi_label = True,
                from_logits = True,
                num_labels = NUM_CLASSES
            ),
            keras.metrics.Recall(thresholds = 0),
            keras.metrics.Precision(thresholds = 0)
        ]
    )

In [35]:
with strategy.scope():
    
    csv_logger = keras.callbacks.CSVLogger(os.path.join(SAVE_DIR, 'weighted_crossentropy_loss_unfreeze_last_three.csv'))
    tensorboard = keras.callbacks.TensorBoard(os.path.join(SAVE_DIR, 'logs', 'weighted_crossentropy_loss_unfreeze_last_three/'))
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(SAVE_DIR, "weighted-crossentropy-loss-unfreeze-last-three-at-epoch-{epoch:02d}.hdf5"))
    reducelr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=.4, patience=4, min_lr=1e-7)

In [36]:
model.fit(
    train_ds,
    epochs = EPOCHS,
    class_weight = class_weight,
    callbacks = [csv_logger, tensorboard, checkpoint, reducelr],
    validation_data = val_ds
)

Epoch 1/40


2023-06-15 04:17:21.142365: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_78/ReadVariableOp.
2023-06-15 04:17:21.629678: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_78/ReadVariableOp.


138/139 [============================>.] - ETA: 0s - loss: 0.5675 - accuracy: 0.1824 - auc: 0.5132 - recall: 0.7154 - precision: 0.1568

2023-06-15 04:18:27.836483: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2023-06-15 04:18:28.048961: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


139/139 [==============================] - 104s 388ms/step - loss: 0.5672 - accuracy: 0.1817 - auc: 0.5133 - recall: 0.7151 - precision: 0.1569 - val_loss: 2.0572 - val_accuracy: 0.0100 - val_auc: 0.6199 - val_recall: 0.6668 - val_precision: 0.2115 - lr: 3.5000e-04
Epoch 2/40
139/139 [==============================] - 17s 112ms/step - loss: 0.5410 - accuracy: 0.3480 - auc: 0.6194 - recall: 0.4287 - precision: 0.1582 - val_loss: 1.5453 - val_accuracy: 0.5047 - val_auc: 0.6882 - val_recall: 0.4100 - val_precision: 0.2550 - lr: 3.5000e-04
Epoch 3/40
139/139 [==============================] - 18s 113ms/step - loss: 0.5003 - accuracy: 0.4793 - auc: 0.6991 - recall: 0.2607 - precision: 0.1783 - val_loss: 1.5952 - val_accuracy: 0.5303 - val_auc: 0.7766 - val_recall: 0.1903 - val_precision: 0.1495 - lr: 3.5000e-04
Epoch 4/40
139/139 [==============================] - 17s 112ms/step - loss: 0.4620 - accuracy: 0.5001 - auc: 0.7520 - recall: 0.1955 - precision: 0.1641 - val_loss: 1.3428 - val_acc

In [37]:
model.save(os.path.join(SAVE_DIR, 'weighted_crossentropy_loss_unfreeze_last_three'))

INFO:tensorflow:Assets written to: /kaggle/working/weighted_crossentropy_loss_unfreeze_last_three/assets


INFO:tensorflow:Assets written to: /kaggle/working/weighted_crossentropy_loss_unfreeze_last_three/assets
